In [1]:
import pandas as pd
import numpy as np
import sklearn
from gensim import corpora
from gensim import models
from gensim import similarities
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import heapq

In [2]:
df_ingre = pd.read_csv('recipe_dataset/recipe_ingre.csv')
df_basic = pd.read_csv('recipe_dataset/recipe_basic.csv', encoding="cp949")

df_tmp=df_ingre[['RECIPE_ID','IRDNT_NM']]
df_tmp_arr=[[]for i in range(538)]
num=0
for i in df_tmp['IRDNT_NM'] :
    df_tmp_arr[df_tmp['RECIPE_ID'][num]].append(i)
    num+=1
    
df_tmp_arr.pop(0)
num=0
for i in df_tmp_arr :
    df_tmp_arr[num] = list(set(df_tmp_arr[num]))
    num+=1
    
ingre_dict = {}

for i in range(1, 538, 1):
    ingre_dict[str(i)] = df_tmp_arr[i-1]

In [3]:
len(ingre_dict)

537

In [4]:
bags_of_words = [ Counter(IRDNT_NM) for IRDNT_NM in df_tmp_arr ]

In [5]:
len(bags_of_words)

537

In [6]:
#Find sum of every ingredient using Counter()
sumbags = sum(bags_of_words, Counter())


# Finally, plot the 10 most used ingredients
clean_df = pd.DataFrame.from_dict(sumbags, orient='index').reset_index()
clean_df = clean_df.rename(columns={'index':'ingredient', 0:'count'})

top_ing = clean_df.sort_values('count', ascending=False)

In [7]:
top_ing

,ingredient,count
3,소금,332
14,간장,249
5,설탕,214
24,후춧가루,210
11,다진마늘,203
...,...,...
338,열무김치,1
339,이스트,1
341,아욱,1
342,해삼,1


In [8]:
ingr_only_dict = clean_df['ingredient'].to_dict()

In [9]:
#Get all clean ingredients in list format per recipe
ingr_list = []
for IRDNT_NM in df_tmp_arr:
    ingr_list.append(IRDNT_NM)

In [10]:
len(ingr_list)

537

In [11]:
##Get clean ingredients to be used as an input for word2vec model to identify ingredient similarity.

ingr_clean_df = pd.DataFrame({'Ingredient':ingr_list})

In [12]:
#Create a dictionary for all the ingredients in the recipe list

dictionary = corpora.Dictionary(ingr_list)

In [13]:
#Applying doc2bow on the dictionary of ingredients, which converts the ingredient to a number in every recipe
#This input format is needed for TfIdfmodel
bow_corpus = [dictionary.doc2bow(text) for text in ingr_list]

In [14]:
# train the model
tfidf = models.TfidfModel(bow_corpus)

In [15]:
corpus_tfidf = tfidf[bow_corpus]

In [16]:
#Use similarities library from gensim to get the cosine similarity of the tfidf results

index = similarities.MatrixSimilarity(tfidf[bow_corpus])
index.save('ingr.index')
index = similarities.MatrixSimilarity.load('ingr.index')

sims = index[corpus_tfidf]
sims_list = [(i,j) for i,j in enumerate(sims)]

#Creating a list to hold the cosine similarity results for tfidf
tf_idf_list = []

for i,j in enumerate(sims_list):
    tf_idf_list.append(sims_list[i][1])

In [17]:
#Create recipe dict- to be used in creating dataframe in next step - used to decode recipe id
recipe_dict = {k: v for k, v in enumerate(df_basic.RECIPE_NM_KO)}

In [18]:
#Use cosine similarity results to get the top 10 similar recipes for every recipe.
tf_idf_top  = []
similar_recipes_df = pd.DataFrame([])
same_item = []

#Get only top 11 largest values from the tf_idf_list - 1 recipe will be the same as itself (hence 12)
for i,item in enumerate(tf_idf_list):
    tf_idf_top.append(heapq.nlargest(11,enumerate(item), key=lambda x: x[1]))

#Remove the recipe value with 1.0 similarity - since it is the same recipe
for i,list_item in enumerate(tf_idf_top):
    for j,k in enumerate(list_item):
        if tf_idf_top[i][j][1] != 1.0:
            similar_recipes_df = similar_recipes_df.append(pd.DataFrame({'Similar_Recipe_ID': recipe_dict.get(tf_idf_top[i][j][0]),'TF-IDF Value': tf_idf_top[i][j][1],'Recipe_ID': recipe_dict.get(i)}, index=[0]), ignore_index=True)

In [19]:
similar_recipes_df = similar_recipes_df[similar_recipes_df['Recipe_ID'] != similar_recipes_df['Similar_Recipe_ID']]

In [20]:
#Create cosine similarity matrix for all recipes 27637*27637
#Since this is a huge matrix, the top 10 similar recipe logic is a better option.

names = [i for i in range(1,len(tf_idf_list))]
final_df = pd.DataFrame.from_dict(zip(names,tf_idf_list))

In [21]:
final_df

,0,1
0,1,"[1.0000001, 0.0017785624, 0.11558835, 0.333323..."
1,2,"[0.0017785624, 1.0, 0.0024592949, 0.002959687,..."
2,3,"[0.11558835, 0.0024592949, 1.0, 0.04783988, 0...."
3,4,"[0.3333232, 0.002959687, 0.04783988, 1.0, 0.00..."
4,5,"[0.013954182, 0.12799355, 0.007930435, 0.00954..."
...,...,...
531,532,"[0.12392111, 0.0016378999, 0.2967598, 0.020259..."
532,533,"[0.0017090708, 0.0016938464, 0.002363206, 0.00..."
533,534,"[0.030211665, 0.0, 0.093179785, 0.014064255, 0..."
534,535,"[0.068171576, 0.0029544362, 0.0041219443, 0.00..."


In [22]:
df_basic['RECIPE_ID'] = df_basic.index+1
recipe_name_df = df_basic[['RECIPE_ID','RECIPE_NM_KO']]
final_df['RECIPE_ID'] = final_df.index+1

recipe_tf_idf_df = final_df.merge(recipe_name_df,how='left', left_on='RECIPE_ID', right_on='RECIPE_ID')

In [23]:
recipe_tf_idf_df

,0,1,RECIPE_ID,RECIPE_NM_KO
0,1,"[1.0000001, 0.0017785624, 0.11558835, 0.333323...",1,나물비빔밥
1,2,"[0.0017785624, 1.0, 0.0024592949, 0.002959687,...",2,오곡밥
2,3,"[0.11558835, 0.0024592949, 1.0, 0.04783988, 0....",3,잡채밥
3,4,"[0.3333232, 0.002959687, 0.04783988, 1.0, 0.00...",4,콩나물밥
4,5,"[0.013954182, 0.12799355, 0.007930435, 0.00954...",5,약식
...,...,...,...,...
531,532,"[0.12392111, 0.0016378999, 0.2967598, 0.020259...",532,배추만두
532,533,"[0.0017090708, 0.0016938464, 0.002363206, 0.00...",533,식빵고구마파이
533,534,"[0.030211665, 0.0, 0.093179785, 0.014064255, 0...",534,카레토마토달걀볶음밥
534,535,"[0.068171576, 0.0029544362, 0.0041219443, 0.00...",535,쪽파 새우강회


In [24]:
#Create a list from tfidf results
#This will be used to identify ingredient importance within every recipe

corpus_list = []
for doc in corpus_tfidf:
    corpus_list.append(doc)

In [25]:
#Create a flat list to eliminate repetition of ingredients and create a dict to hold the results

flat_list = []
for sublist in ingr_list:
    for item in sublist:
        if item not in flat_list:
            flat_list.append(item)
ing_dict =  {k: v for k, v in enumerate(flat_list)}

In [26]:
#Create a dataframe with tf-idf values per ingredient for every recipe.
corpus_df = pd.DataFrame([])

for i,list_item in enumerate(corpus_list):
    for j,k in enumerate(list_item):
        corpus_df = corpus_df.append(pd.DataFrame({'Ingredient': dictionary.get(corpus_list[i][j][0]),'TF-IDF Value': corpus_list[i][j][1],'Recipe_ID': i}, index=[0]), ignore_index=True)

In [27]:
df_basic['RECIPE_ID'] = df_basic.index
recipe_tf_idf_df = corpus_df.merge(df_basic,how='left', left_on='Recipe_ID', right_on='RECIPE_ID')
recipe_tf_idf_df = recipe_tf_idf_df[['Recipe_ID','RECIPE_NM_KO','Ingredient','TF-IDF Value']]

In [28]:
for i, j in enumerate(recipe_tf_idf_df["RECIPE_NM_KO"].drop_duplicates()):
    test = recipe_tf_idf_df[recipe_tf_idf_df["RECIPE_NM_KO"] == j]
    print(test.sort_values(by=['TF-IDF Value'], axis=0, ascending=False))

    Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
8           0        나물비빔밥          묵      0.492708
2           0        나물비빔밥        고사리      0.360205
13          0        나물비빔밥         숙주      0.350923
6           0        나물비빔밥        도라지      0.342526
14          0        나물비빔밥          쌀      0.299141
16          0        나물비빔밥        콩나물      0.270201
9           0        나물비빔밥        미나리      0.231026
7           0        나물비빔밥       돼지고기      0.207393
3           0        나물비빔밥        고추장      0.180763
1           0        나물비빔밥         계란      0.156383
12          0        나물비빔밥        쇠고기      0.154467
5           0        나물비빔밥        다진파      0.138880
15          0        나물비빔밥        참기름      0.087011
4           0        나물비빔밥       다진마늘      0.085699
10          0        나물비빔밥         설탕      0.081050
0           0        나물비빔밥         간장      0.067706
11          0        나물비빔밥         소금      0.042362
    Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
20          

364         37         멸치볶음         간장      0.155226
     Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
384         38        갈치무조림         갈치      0.656124
380         38        갈치무조림         감자      0.360735
383         38        갈치무조림         생강      0.281498
382         38        갈치무조림          무      0.277210
373         38        갈치무조림        고추장      0.259533
379         38        갈치무조림         마늘      0.215154
377         38        갈치무조림       고춧가루      0.187447
381         38        갈치무조림         대파      0.181375
376         38        갈치무조림         고추      0.178445
378         38        갈치무조림        깨소금      0.176528
375         38        갈치무조림        참기름      0.124926
374         38        갈치무조림         설탕      0.116369
372         38        갈치무조림         간장      0.097209
     Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
393         39         닭불고기        닭고기      0.524971
392         39         닭불고기        생강즙      0.496361
386         39         닭불고기        고추장      0.

672         62         열무김치         소금      0.063465
     Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
685         63        오이소박이         부추      0.537043
688         63        오이소박이       다진생강      0.522417
687         63        오이소박이         오이      0.383023
689         63        오이소박이         청주      0.292932
682         63        오이소박이        다진파      0.264990
686         63        오이소박이       고춧가루      0.249107
681         63        오이소박이       다진마늘      0.163517
683         63        오이소박이         설탕      0.154647
680         63        오이소박이         간장      0.129185
684         63        오이소박이         소금      0.080829
     Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
698         64         나박김치         배추      0.530954
690         64         나박김치        미나리      0.435810
697         64         나박김치         생강      0.369854
696         64         나박김치          무      0.364220
694         64         나박김치         마늘      0.282686
693         64         나박김치       고춧가루      0.

1114        106          불고기         간장      0.128529
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
1126        107          부추전         부추      0.701000
1127        107          부추전        밀가루      0.499958
1123        107          부추전         계란      0.389480
1125        107          부추전         고추      0.309541
1124        107          부추전         소금      0.105505
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
1136        108       돼지갈비구이         상추      0.496398
1135        108       돼지갈비구이         깻잎      0.444197
1138        108       돼지갈비구이        생강즙      0.409952
1131        108       돼지갈비구이       돼지고기      0.349719
1137        108       돼지갈비구이         청주      0.258884
1130        108       돼지갈비구이        다진파      0.234189
1133        108       돼지갈비구이       고춧가루      0.220152
1134        108       돼지갈비구이        깨소금      0.207328
1129        108       돼지갈비구이       다진마늘      0.144511
1132        108       돼지갈비구이         설탕      0.136672
1128        108       돼지갈비구이

      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
1597        151          빈대떡       석이버섯      0.453479
1598        151          빈대떡         녹두      0.453479
1596        151          빈대떡        실고추      0.378425
1589        151          빈대떡         숙주      0.368674
1595        151          빈대떡      느타리버섯      0.304521
1594        151          빈대떡       올리브유      0.287647
1587        151          빈대떡       돼지고기      0.217884
1586        151          빈대떡        다진파      0.145906
1593        151          빈대떡         대파      0.132717
1592        151          빈대떡        깨소금      0.129170
1590        151          빈대떡        참기름      0.091412
1585        151          빈대떡       다진마늘      0.090034
1591        151          빈대떡       후춧가루      0.086896
1584        151          빈대떡         간장      0.071131
1588        151          빈대떡         소금      0.044505
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
1602        152          화양적        도라지      0.595131
1611        152          화양적

...

3410        322     오징어도라지생채         소금      0.067565
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
3427        323        애호박무침       새우젓국      0.698640
3423        323        애호박무침          파      0.397661
3426        323        애호박무침        애호박      0.390488
3425        323        애호박무침         피망      0.288650
3424        323        애호박무침        식용유      0.189056
3421        323        애호박무침       고춧가루      0.164710
3422        323        애호박무침        깨소금      0.155116
3420        323        애호박무침        참기름      0.109773
3419        323        애호박무침       다진마늘      0.108118
3418        323        애호박무침         간장      0.085418
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
3430        324       양송이버섯죽         찹쌀      0.476886
3437        324       양송이버섯죽      양송이버섯      0.459425
3432        324       양송이버섯죽         호박      0.430246
3436        324       양송이버섯죽         깻잎      0.391003
3435        324       양송이버섯죽         실파      0.293295
3431        324       양송이버섯죽

3805        366         깐풍새우         간장      0.106871
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
3830        367      해산물두부수프         게살      0.440382
3828        367      해산물두부수프        생선살      0.408455
3821        367      해산물두부수프         전분      0.385803
3822        367      해산물두부수프       계란흰자      0.385803
3829        367      해산물두부수프        다시다      0.368233
3827        367      해산물두부수프         육수      0.238417
3823        367      해산물두부수프         새우      0.215012
3824        367      해산물두부수프        오징어      0.210636
3826        367      해산물두부수프         두부      0.179422
3825        367      해산물두부수프         청주      0.137221
3820        367      해산물두부수프        참기름      0.077770
3819        367      해산물두부수프         소금      0.037863
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
3834        368         채소스프      쇠고기육수      0.521771
3839        368         채소스프      토마토소스      0.470390
3836        368         채소스프        샐러리      0.359887
3837        368         채소스프

4203        399     오징어볶음과소면         간장      0.118710
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
4227        400         부추잡채        굴소스      0.510710
4222        400         부추잡채         부추      0.456042
4226        400         부추잡채       다진생강      0.443622
4218        400         부추잡채       돼지고기      0.336030
4224        400         부추잡채        식용유      0.242802
4221        400         부추잡채         고추      0.201375
4223        400         부추잡채        깨소금      0.199212
4225        400         부추잡채         양파      0.196028
4220        400         부추잡채        참기름      0.140980
4217        400         부추잡채       다진마늘      0.138855
4219        400         부추잡채         소금      0.068638
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
4241        401         쟁반라면         라면      0.501195
4242        401         쟁반라면        치커리      0.419331
4238        401         쟁반라면         겨자      0.406429
4240        401         쟁반라면         상추      0.322855
4239        401         쟁반라면

4663        445       깻잎말이김치         설탕      0.100622
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
4679        446       붉은갓동치미        붉은갓      0.608839
4680        446       붉은갓동치미       삭힌고추      0.608839
4676        446       붉은갓동치미          배      0.346547
4678        446       붉은갓동치미         생강      0.215559
4677        446       붉은갓동치미          무      0.212276
4674        446       붉은갓동치미         마늘      0.164756
4675        446       붉은갓동치미         양파      0.133017
4673        446       붉은갓동치미         소금      0.046575
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
4690        447      파프리카볶음밥         찬밥      0.548426
4689        447      파프리카볶음밥       파프리카      0.425458
4683        447      파프리카볶음밥         호박      0.368281
4688        447      파프리카볶음밥       올리브유      0.347873
4685        447      파프리카볶음밥         감자      0.319227
4687        447      파프리카볶음밥         피망      0.290696
4682        447      파프리카볶음밥         당근      0.193877
4686        447      파프리카볶음밥

      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
5143        493        미소된장국       미소된장      0.711484
5142        493        미소된장국       팽이버섯      0.410120
5139        493        미소된장국        다시마      0.349520
5141        493        미소된장국         두부      0.312534
5140        493        미소된장국         실파      0.307635
5138        493        미소된장국         간장      0.105411
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value
5158        494         비빔냉면      샐러드채소      0.558508
5153        494         비빔냉면         냉면      0.460897
5157        494         비빔냉면      방울토마토      0.385615
5155        494         비빔냉면         육수      0.269028
5152        494         비빔냉면         물엿      0.265674
5156        494         비빔냉면          무      0.194727
5146        494         비빔냉면        고추장      0.182310
5154        494         비빔냉면         식초      0.179771
5145        494         비빔냉면         계란      0.157720
5148        494         비빔냉면        다진파      0.140068
5151        494         비빔냉면

In [29]:
test_1 = recipe_tf_idf_df[["RECIPE_NM_KO", "Ingredient", "TF-IDF Value"]]

In [31]:
test_2 = df_ingre[["RECIPE_NM_KO", "IRDNT_NM", "SCORE"]]

In [33]:
t_list = []

for r_id, r_menu in enumerate(test_1["RECIPE_NM_KO"]):
    for m_id, m_menu in enumerate(test_2["RECIPE_NM_KO"]):
        if r_menu == m_menu:
            if test_2["IRDNT_NM"][m_id] == test_1["Ingredient"][r_id]:
                t_list.append(test_2["SCORE"][m_id])
                break

In [35]:
recipe_tf_idf_df.insert(4, 'SCORE',df_ingre['SCORE'])

In [36]:
tfidf_weight = pd.DataFrame(recipe_tf_idf_df['TF-IDF Value'].mul(recipe_tf_idf_df['SCORE'], axis=0, level=None, fill_value=None))

In [37]:
recipe_tf_idf_df.insert(5, 'TF-IDF_WEIGHT',tfidf_weight)

In [39]:
recipe_tf_idf_df.to_csv('tfidf_weight.csv', encoding="utf-8-sig")

In [40]:
for i, j in enumerate(recipe_tf_idf_df["RECIPE_NM_KO"].drop_duplicates()):
    test = recipe_tf_idf_df[recipe_tf_idf_df["RECIPE_NM_KO"] == j]
    print(test.sort_values(by=['TF-IDF_WEIGHT'], axis=0, ascending=False))

    Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value  SCORE  TF-IDF_WEIGHT
2           0        나물비빔밥        고사리      0.360205      5       1.801023
13          0        나물비빔밥         숙주      0.350923      5       1.754613
14          0        나물비빔밥          쌀      0.299141      5       1.495705
6           0        나물비빔밥        도라지      0.342526      4       1.370105
16          0        나물비빔밥        콩나물      0.270201      5       1.351006
9           0        나물비빔밥        미나리      0.231026      4       0.924104
7           0        나물비빔밥       돼지고기      0.207393      4       0.829573
3           0        나물비빔밥        고추장      0.180763      3       0.542290
8           0        나물비빔밥          묵      0.492708      1       0.492708
5           0        나물비빔밥        다진파      0.138880      2       0.277761
15          0        나물비빔밥        참기름      0.087011      3       0.261032
0           0        나물비빔밥         간장      0.067706      3       0.203117
4           0        나물비빔밥       다진마늘 

223         22          생태국         소금      0.062943      2       0.125886
     Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value  SCORE  TF-IDF_WEIGHT
236         23      연어까르파치오      양송이버섯      0.245014      5       1.225072
237         23      연어까르파치오         연어      0.390028      3       1.170085
239         23      연어까르파치오        차이브      0.390028      3       1.170085
233         23      연어까르파치오        레몬즙      0.313414      3       0.940243
240         23      연어까르파치오      페스토소스      0.438366      2       0.876732
234         23      연어까르파치오       모듬채소      0.438366      1       0.438366
235         23      연어까르파치오      방울토마토      0.302664      1       0.302664
238         23      연어까르파치오       올리브유      0.216739      1       0.216739
230         23      연어까르파치오         소금      0.033534      3       0.100602
232         23      연어까르파치오         양파      0.095773      1       0.095773
231         23      연어까르파치오       후춧가루      0.065475      1       0.065475
     Recipe_ID RECIPE_NM_

395         40      두부다시마말이         소금      0.054038      1       0.054038
     Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value  SCORE  TF-IDF_WEIGHT
409         41         팥칼국수        칼국수      0.772761      5       3.863807
408         41         팥칼국수          팥      0.613216      5       3.066079
406         41         팥칼국수         설탕      0.145102      3       0.435306
407         41         팥칼국수         소금      0.075840      2       0.151680
     Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value  SCORE  TF-IDF_WEIGHT
419         42        콩나물잡채        콩나물      0.462777      4       1.851108
423         42        콩나물잡채        깨소금      0.210580      5       1.052899
417         42        콩나물잡채        쇠고기      0.264558      3       0.793675
414         42        콩나물잡채        미나리      0.395681      2       0.791363
413         42        콩나물잡채        다진파      0.237862      3       0.713587
418         42        콩나물잡채        참기름      0.149024      3       0.447073
422         42        콩나물

721         67          동치미         마늘      0.197455      1       0.197455
     Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value  SCORE  TF-IDF_WEIGHT
735         68          갓김치          갓      0.506591      3       1.519774
731         68          갓김치        밀가루      0.274417      3       0.823252
734         68          갓김치         생강      0.268021      3       0.804064
730         68          갓김치         마늘      0.204853      3       0.614560
736         68          갓김치        멸치젓      0.590066      1       0.590066
728         68          갓김치       고춧가루      0.178473      3       0.535419
729         68          갓김치        깨소금      0.168077      2       0.336153
727         68          갓김치         소금      0.057910      5       0.289549
732         68          갓김치         실파      0.270116      1       0.270116
733         68          갓김치          무      0.263939      1       0.263939
     Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value  SCORE  TF-IDF_WEIGHT
749         69         꽃게

948         87          백김치         소금      0.042968      1       0.042968
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value  SCORE  TF-IDF_WEIGHT
964          88        깻잎장아찌          파      0.500806      3       1.502417
965          88        깻잎장아찌         깻잎      0.418533      3       1.255600
5431        524        깻잎장아찌         통깨      0.409338      3       1.228014
963          88        깻잎장아찌         마늘      0.238093      5       1.190467
967          88        깻잎장아찌        실고추      0.572306      2       1.144612
5432        524        깻잎장아찌      까나리액젓      0.645325      1       0.645325
961          88        깻잎장아찌       고춧가루      0.207432      3       0.622297
5433        524        깻잎장아찌         쪽파      0.591923      1       0.591923
5430        524        깻잎장아찌         고추      0.185812      3       0.557437
960          88        깻잎장아찌         간장      0.107573      3       0.322720
966          88        깻잎장아찌         생강      0.311511      1       0.311511
5429        5

...

      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value  SCORE  TF-IDF_WEIGHT
3296        310      버섯청국장찌개        청국장      0.592398      4       2.369591
3294        310      버섯청국장찌개      양송이버섯      0.372142      5       1.860711
3295        310      버섯청국장찌개      느타리버섯      0.348507      4       1.394027
3293        310      버섯청국장찌개         두부      0.241357      4       0.965430
3292        310      버섯청국장찌개         대파      0.151887      5       0.759433
3290        310      버섯청국장찌개       표고버섯      0.230390      3       0.691171
3289        310      버섯청국장찌개         고추      0.149433      4       0.597731
3291        310      버섯청국장찌개         호박      0.348507      1       0.348507
3297        310      버섯청국장찌개       팽이버섯      0.316720      1       0.316720
3288        310      버섯청국장찌개       다진마늘      0.103039      1       0.103039
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value  SCORE  TF-IDF_WEIGHT
3300        311        부추부침개         부추      0.472571      5       2.362853
3304        

      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value  SCORE  TF-IDF_WEIGHT
3723        357     삼색태극말이초밥          밥      0.261465      5       1.307324
3725        357     삼색태극말이초밥      계란노른자      0.410030      3       1.230090
3726        357     삼색태극말이초밥      파슬리가루      0.352393      2       0.704786
3728        357     삼색태극말이초밥        오보로      0.662907      1       0.662907
3724        357     삼색태극말이초밥         식초      0.213374      2       0.426749
3727        357     삼색태극말이초밥          김      0.377322      1       0.377322
3722        357     삼색태극말이초밥         소금      0.050711      5       0.253553
3721        357     삼색태극말이초밥         설탕      0.097023      1       0.097023
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value  SCORE  TF-IDF_WEIGHT
3741        358       캘리포니아롤       아보카도      0.408439      3       1.225318
3735        358       캘리포니아롤         통깨      0.201942      5       1.009710
3738        358       캘리포니아롤         게살      0.363401      2       0.726803
3737        

      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value  SCORE  TF-IDF_WEIGHT
4371        414    고등어튀김케첩조림        고등어      0.481800      3       1.445400
4364        414    고등어튀김케첩조림         마늘      0.194828      4       0.779314
4367        414    고등어튀김케첩조림         실파      0.256897      3       0.770691
4366        414    고등어튀김케첩조림         피망      0.297462      2       0.594925
4363        414    고등어튀김케첩조림         설탕      0.105375      5       0.526876
4370        414    고등어튀김케첩조림         생강      0.254905      2       0.509810
4372        414    고등어튀김케첩조림         케첩      0.481800      1       0.481800
4365        414    고등어튀김케첩조림         양파      0.157297      3       0.471890
4369        414    고등어튀김케첩조림         육수      0.346802      1       0.346802
4368        414    고등어튀김케첩조림         녹말      0.338314      1       0.338314
4362        414    고등어튀김케첩조림         간장      0.088026      2       0.176051
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value  SCORE  TF-IDF_WEIGHT
4377        

      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value  SCORE  TF-IDF_WEIGHT
4780        460        전어회무침        콩가루      0.413298      5       2.066492
4781        460        전어회무침         통깨      0.262160      3       0.786481
4782        460        전어회무침         전어      0.530235      1       0.530235
4783        460        전어회무침       전어젓갈      0.530235      1       0.530235
4778        460        전어회무침         식초      0.170670      3       0.512011
4773        460        전어회무침        미나리      0.221207      2       0.442414
4777        460        전어회무침         양파      0.115844      3       0.347532
4772        460        전어회무침       다진마늘      0.082057      3       0.246170
4774        460        전어회무침         설탕      0.077605      3       0.232816
4779        460        전어회무침          무      0.184870      1       0.184870
4771        460        전어회무침        고추장      0.173081      1       0.173081
4776        460        전어회무침       고춧가루      0.125007      1       0.125007
4775        

5342        513   토마토미니새송이볶음         소금      0.046319      3       0.138958
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value  SCORE  TF-IDF_WEIGHT
5353        514    닭가슴살 호박말이     허니머스타드      0.520763      5       2.603813
5349        514    닭가슴살 호박말이         호박      0.330310      5       1.651552
5354        514    닭가슴살 호박말이         노각      0.631052      2       1.262105
5352        514    닭가슴살 호박말이       마요네즈      0.352712      3       1.058135
5351        514    닭가슴살 호박말이        닭고기      0.293009      3       0.879027
5350        514    닭가슴살 호박말이       후춧가루      0.094255      3       0.282766
5348        514    닭가슴살 호박말이         소금      0.048274      1       0.048274
      Recipe_ID RECIPE_NM_KO Ingredient  TF-IDF Value  SCORE  TF-IDF_WEIGHT
5360        515       고구마잎무침       고구마잎      0.925815      1       0.925815
5355        515       고구마잎무침       다진마늘      0.143275      5       0.716374
5359        515       고구마잎무침        깨소금      0.205554      3       0.616662
5358        